In [ ]:
# import settings and functions
%run ./../../imports.ipynb


## What mesh?

Copy your choice to the next cell

for SquareTop:
```
analytical_solution_tag = "-ana_square_top"
generate_config = generateConfig_squareTop
generate_mesh = generateMesh_squareTop
```

for SquareSinCos:
```
analytical_solution_tag = "-ana_square_sincos"
generate_config = generateConfig_squareSinCos
generate_mesh = generateMesh_squareSinCos
```

In [ ]:
# Change according to instruction above
analytical_solution_tag = "-ana_mexi_hat"
generate_config = generateConfig_squareMexiHat
generate_mesh = generateMesh_squareMexiHat

# analytical_solution_tag = "-ana_square_top"
# generate_config = generateConfig_squareTop
# generate_mesh = generateMesh_squareTop

# analytical_solution_tag = "-ana_square_sincos"
# generate_config = generateConfig_squareSinCos
# generate_mesh = generateMesh_squareSinCos

# analytical_solution_tag = "-ana_L_shape"
# generate_config = generateConfig_Lshape
# generate_mesh = generateMesh_Lshape

## Analysis setup

In [ ]:
# prefix = "c5_hat_missing_"

# which executable?

exe = data_driven_diffusion_snes
sumanalys = "sumanalys.csv"
ana_name = "ana_square_mexi_mixed_order"

ana_compare_exe = [data_driven_diffusion_snes, classic_diffusion]
ana_compare_name = ["ana_square_mexi_dd", "ana_square_mexi_classic"]
# ana_compare_name = ["ana_square_mexi_mixed"]
ana_compare_sum = ["sumanalys.csv", "sumanalys.csv", "FEM_errors.csv", "sumanalys.csv"]

# Convergence analysis parameters
order_list = [1, 2, 3] # approximation order p
elem_size_list = [0.08, 0.04, 0.02] # element size h
# elem_size_list = [0.1, 0.05, 0.025] # element size h
# order_list = [2, 3] # approximation order p
# elem_size_list = [1./3., 1./7., 1./13.] # element size h
params.triangle_mesh = True
params.nproc = 1 # number of processors
jumps = ""
if params.nproc == 1:
    jumps = "-get_jumps"
# jumps = "-get_jumps"

run_test = True
run_analysis = True

# params.triangle_mesh = False

run_test = False
run_analysis = False

# naming = ["order", "gaussnum", "iterations","volume", "datanum","rmsPoiErr", "errorEstimator",
#           "L2norm", "H1seminorm","fluxErr", "orderRefinementCounter", "errorIndicatorGrad", "errorIndicatorDiv", "jumpL2", "jumpHdiv", "eleNum"]
naming = ["order", "gaussnum", "iterations","volume", "datanum","rmsPoiErr", "errorEstimator",
          "L2norm", "H1seminorm","fluxErr", "orderRefinementCounter"]

error_name_list = ["L2norm", "H1seminorm", "fluxErr"]
error_label_list = [(r'Global error $L^2$-norm'),
               (r'Global error $H^1$-seminorm'), (r'Global Flux error')]

In [ ]:
params.conductivity = 1.0 # linear conductivity
# params.element_size = elem_size_list[0] # element size in the regular mesh
params.element_size = 0.05
params.order = 2 # approximation order for displacements

# params.triangle_mesh = False # use triangular mesh

# Pre-processing parameters
# params.mesh_file = "square_mexi"
params.mesh_file = "L_shape"
params.length_x = 1
params.length_y = 1
params.length_z = 0
params.show_mesh = True


# solution parameters
params.log_file = "log" # log file name 


## Run test

In [ ]:
# start display for showing results
display = Display(backend="xvfb", visible=False, size=(1024, 768))
display.start()

In [ ]:
# Testing mesh generation
if run_test:
    params.show_mesh = True
    generate_config(params)
    generate_mesh(params)

# stop the display
display.stop()


In [ ]:
params.part_file = params.mesh_file + "_" + str(params.nproc) + "p.h5m"
!{mofem_part} -my_file {params.mesh_file + ".h5m"} -my_nparts {params.nproc} -output_file {params.part_file} -dim 2 -adj_dim 1

In [ ]:
# recreate dataset
recreate_dataset = True
csv_name = "dummy_tree"

dummy_noise = 0.1

dummy_count = 10000

if recreate_dataset:
    !{create_csv_dataset} -output_file {csv_name}.csv -my_dummy_noise_q 0 -my_dummy_noise_k {dummy_noise} -my_dummy_k 1 -my_dummy_range_dp 9.0 -my_dummy_count {int(dummy_count)}

    scal = np.genfromtxt('./scaling.in')
    print(scal)

data = pd.read_csv(csv_name+".csv", index_col=False)
headers = []
headers = data.columns.values
print(headers)

In [ ]:
noise_list = [0.0, 0.01, 0.02, 0.05, 0.1]
# noise_list = [0.02, 0.05, 0.1]
ana_name = "noise_sincos_"

ana_name_list = []

for noise in noise_list:
    ana_name_list.append(ana_name + str(noise))
    
# dummy_count_list = [1e3, 1e4, 1e5, 1e6, 1e7]
dummy_count_list = [1e3, 1e4, 1e5, 1e6]

realisations = 30

In [ ]:
if run_analysis:
    !rm sumanalys.csv # making sure the previous analysis doesn't affect this one

    # loop over noise_lisgt and ana_name_list
    for (noise_in, ana_csv_name) in zip(noise_list, ana_name_list):
        for dummy_count in dummy_count_list:
#             !{create_csv_dataset} -output_file {csv_name}.csv -my_dummy_noise_q 0 -my_dummy_noise_k {noise_in} -my_dummy_k 1 -my_dummy_range_dp 9.0 -my_dummy_count {int(dummy_count)}

            for i in range(realisations):
#                 recreate dataset for each realisation
                !{create_csv_dataset} -output_file {csv_name}.csv -my_dummy_noise_q 0 -my_dummy_noise_k {noise_in} -my_dummy_k 1 -my_dummy_range_dp 9.0 -my_dummy_count {int(dummy_count)}
#                 run analysis
                !{exe} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag} -csv_tree_file {csv_name}.csv -write_long_error_file -rand_ini 

        !mv ./sumanalys.csv ./{ana_csv_name}.csv


In [ ]:
ana_df_list = []

for ana_name in ana_name_list:
    ana_df_list.append(pd.read_csv(f'{ana_name}.csv', header=None, names=naming,  index_col=False))


In [ ]:
print(ana_df_list[0].head())

# print headers
print(ana_df_list[0].columns.values)

In [ ]:
def plot_dd_ana(ana_df_list, noise_list, errorType='L2norm', grouping='mean', label = "", box_plots=0, yscale_log=False):
    # plt.ioff()  # Turn off interactive plotting
    first_point = 1
    grouped_data_list = []
    for a in range(0, len(ana_df_list)):

        if grouping == 'mean':
            grouped_data_list.append(ana_df_list[a].groupby(['datanum']).mean())
            plot_title = 'Mean'
        if grouping == 'std':
            grouped_data_list.append(ana_df_list[a].groupby(['datanum']).std())
            plot_title = 'Standard Deviation'
        if grouping == 'var':
            grouped_data_list.append(ana_df_list[a].groupby(['datanum']).var())
            plot_title = 'Variance'
        if grouping == 'median':
            grouped_data_list.append(ana_df_list[a].groupby(['datanum']).median())
            plot_title = 'Median'

        if box_plots == 1:
            fig = plt.figure()
            ax1 = sns.boxplot(x="datanum", y=ana_df_list[a][errorType], data=ana_df_list[a]).set_title(errorType + ', noise = ' + str(noise_list[a]))
            if yscale_log == True:
                plt.yscale('log')
            plt.savefig('boxplot_' + errorType + '_noise_' + str(noise_list[a]) + '.png')
            plt.show()
            plt.close()

    if box_plots == 1:
        fig = plt.figure()
        for a in range(0, len(ana_df_list)):
            sns.boxplot(x="datanum", y=ana_df_list[a][errorType], data=ana_df_list[a])
        if yscale_log == True:
            plt.yscale('log')
        plt.savefig('boxplot_' + errorType + '.png')
        plt.show()
        plt.close()

    # print(grouped_data_list[0].index)
    # print(grouped_data_list[0].head())
    # print(grouped_data_list[0]["L2norm"][first_point:])

    # how to get L2norm out?
    # print(grouped_data_list[0][errorType])

    fig = plt.figure()
    for a in range(0, len(ana_df_list)):
        # print(grouped_data_list[a].index)
        # print(grouped_data_list[a][errorType])
        # translate index and errorType to numpy arrays with float values
        x = np.array(grouped_data_list[a].index).astype(float)
        y = np.array(grouped_data_list[a][errorType]).astype(float)
        print(x)
        print(y)
        print(noise_list[a])

        plt.plot(x, y, '-x', label = 'noise = ' + str(noise_list[a]))
    plt.xscale('log')
    plt.legend(loc='best')
    if label:        
        plt.ylabel(label)
    else:
        plt.ylabel(errorType)
    plt.xlabel(r'\# data points')
    plt.title(plot_title)
    plt.grid(True)
    if yscale_log == True:
        plt.yscale('log')

    plt.savefig(grouping + '_' + errorType + '.png')
    plt.show()
    # plt.close()
    

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_dd_shaded_ana(ana_df_list, noise_list, errorType='L2norm', label="", yscale_log=False):
    fig = plt.figure()
    plot_title = 'Mean with Standard Deviation'

    for a in range(0, len(ana_df_list)):
        # Group by 'datanum' and calculate mean and std
        grouped_data = ana_df_list[a].groupby(['datanum']).agg([np.mean, np.std])
        x = np.array(grouped_data.index).astype(float)
        y_mean = np.array(grouped_data[(errorType, 'mean')]).astype(float)
        y_std = np.array(grouped_data[(errorType, 'std')]).astype(float)

        # Plot mean
        plt.plot(x, y_mean, '-x', label='noise = ' + str(noise_list[a]))

        # Shade the area for standard deviation
        plt.fill_between(x, y_mean - y_std, y_mean + y_std, alpha=0.5)

    plt.xscale('log')
    if yscale_log:
        plt.yscale('log')
    # legend with fond size 10
    plt.legend(loc='best', prop={'size': 10})
    if label:
        plt.ylabel(label)
    else:
        plt.ylabel(errorType)
    plt.xlabel(r'\# data points')
    plt.title(plot_title)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(errorType + '.png')
    plt.show()
    # plt.close()

In [ ]:
plot_dd_shaded_ana(ana_df_list, noise_list, errorType='L2norm', label=r'Global error $L^2$-norm', yscale_log=True)

plot_dd_shaded_ana(ana_df_list, noise_list, errorType='H1seminorm', label=r'Global error $H^1$-seminorm', yscale_log=True)

plot_dd_shaded_ana(ana_df_list, noise_list, errorType='fluxErr', label=r'Global Flux error', yscale_log=True)

In [ ]:

# naming = ["order", "gaussnum", "iterations","volume", "datanum","rmsPoiErr", "errorEstimator",
#           "L2norm", "H1seminorm","fluxErr", "orderRefinementCounter"]

In [ ]:
# mean
plot_dd_ana(ana_df_list, noise_list, errorType='L2norm', grouping='mean', label = "L2-norm", box_plots=1, yscale_log=True)
# # std
# plot_dd_ana(ana_df_list, noise_list, errorType='L2norm', grouping='std', label = "L2-norm", box_plots=1, yscale_log=False)


plot_dd_ana(ana_df_list, noise_list, errorType='H1seminorm', grouping='mean', label = "H1-seminorm", box_plots=1, yscale_log=True)

plot_dd_ana(ana_df_list, noise_list, errorType='fluxErr', grouping='mean', label = "Flux L2-norm", box_plots=1, yscale_log=True)

## Old bits

In [ ]:
# class MyAnalysis:
#     def __init__(self):
#         self.varName = 'unidentified'
#         d1 = pd.DataFrame()
#         d2 = pd.DataFrame()
#         d3 = pd.DataFrame()
#         d4 = pd.DataFrame()
#         d5 = pd.DataFrame()
#         d6 = pd.DataFrame()
#         d7 = pd.DataFrame()
#         d8 = pd.DataFrame()
#         d9 = pd.DataFrame()
#         self.variable = []
#         self.csvNaming = []
#         self.list = []
#         self.data_list = [d1,d2,d3,d4,d5,d6,d7,d8,d9]

In [ ]:
# noiseAna = MyAnalysis()

# # list analysis name with desired changin variable
# noiseAna.varName = 'noise'
# # noiseAna.list.append(['AS_noiseAna00', 0])
# # noiseAna.list.append(['AS_noiseAna0005', 0.005])
# # noiseAna.list.append(['AS_noiseAna001', 0.01])
# # noiseAna.list.append(['AS_noiseAna005', 0.05])
# noiseAna.list.append(['AS_noiseAna01', 0.1])
# # noiseAna.list.append(['AS_noiseAna02', 0.2])

# # populate the required fields
# for i in range(0,len(noiseAna.list)):
#     noiseAna.csvNaming.append(noiseAna.list[i][0])
#     noiseAna.variable.append(float(noiseAna.list[i][1]))

# #  just a check
# print('variable name: ' + noiseAna.varName)
# print('naming: ' + str(noiseAna.csvNaming))
# print('variables: ' + str(noiseAna.variable))

In [ ]:
# print(exe)

In [ ]:
# if run_analysis:
#     # comment changing variables
#     # in_order = 2
#     # in_k = -3
#     # in_range = 4 # defined above
#     # in_count = 10000
#     # in_noise = 0
    
#     !rm sumanalys.csv # making sure the previous analysis doesn't affect this one

#     for a in range(0,len(noiseAna.variable)):
#         in_noise = noiseAna.variable[a]
#         csvName = noiseAna.csvNaming[a] + '.csv'
#         for i in range(3,5): # 10^1 number of points in the data set
#             dummy_count = 10**(i)
#             !{create_csv_dataset} -output_file {csv_name}.csv -my_dummy_noise_q 0 -my_dummy_noise_k {dummy_noise} -my_dummy_k 1 -my_dummy_range_dp 9.0 -my_dummy_count {int(dummy_count)}
#             for j in range(1,30): # how many realisations
#                 !{exe} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag} -csv_tree_file {csv_name}.csv -write_long_error_file -rand_ini
#         !mv ./sumanalys.csv ./{csvName}

## Postprocessing

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# print(
#     "Loading plot_data(analysis, errorType='rmsErr', grouping='mean', box_plots=0, first_point=0, fig_size=[6.4, 4.8], yscale_log=False, fond_size=15)")

# print("errorType: rmsErr, p_difference, justErr, rmsPoiErr, iterations \ngrouping: mean, std, var \nbox_plots: 1=yes, 0=no \nyscale_log: False, True")


# def plot_dd_ana(analysis, errorType='L2norm', grouping='mean', box_plots=0, first_point=0, fig_size=[6.4, 4.8], yscale_log=False, fond_size=15):
#     grouped_data = [None] * len(analysis.variable)
#     y = [None] * len(analysis.variable)
#     for a in range(0, len(analysis.variable)):

#         if grouping == 'mean':  # taking averages
#             grouped_data[a] = analysis.data_list[a].groupby(['datanum']).mean()
#             plot_title = 'Mean'
#         if grouping == 'std':  # taking standard deviation
#             grouped_data[a] = analysis.data_list[a].groupby(['datanum']).std()
#             plot_title = 'Standard Deviation' # check the terminology
#         if grouping == 'var':  # taking standard deviation
#             grouped_data[a] = analysis.data_list[a].groupby(['datanum']).var()
#             plot_title = 'Variance'
#         if grouping == 'median':  # taking standard deviation
#             grouped_data[a] = analysis.data_list[a].groupby(
#                 ['datanum']).median()
#             plot_title = 'Median'

#         if errorType == 'rmsPoiErr':
#             y[a] = grouped_data[a].rmsPoiErr
#             y_box = analysis.data_list[a].rmsPoiErr
#             y_label = 'Point distance RMS error'
#         # if errorType == 'rmsErrPerc':
#         #     y[a] = grouped_data[a].rmsErrPerc
#         #     y_box = analysis.data_list[a].rmsErrPerc
#         #     y_label = 'Pressure RMS error [%]'
#         if errorType == 'L2norm':
#             y[a] = grouped_data[a].L2norm
#             y_box = analysis.data_list[a].L2norm
#             y_label = 'Temperature RMS error'
#         # if errorType == 'justErr':
#         #     y[a] = grouped_data[a].justErr
#         #     y_box = analysis.data_list[a].justErr
#         #     y_label = 'Accumulated pressure error'
#         # if errorType == 'p_difference':
#         #     y[a] = grouped_data[a].p_difference
#         #     y_box = analysis.data_list[a].p_difference
#         #     y_label = 'Accumulated pressure difference'
#         if errorType == 'iterations':
#             y[a] = grouped_data[a].iterations
#             y_box = analysis.data_list[a].iterations
#             y_label = '# iterations'

#         if box_plots == 1:  # plotting box plots
#             fig = plt.figure(figsize=fig_size)
#             ax1 = sns.boxplot(x="datanum", y=y_box, data=analysis.data_list[a]).set_title(
#                 analysis.varName + ' = ' + str(analysis.variable[a]))
#             plt.show()

#     # fig = plt.figure(figsize=(9, 6))
#     fig = plt.figure(figsize=fig_size)
#     for a in range(0, len(analysis.variable)):
#         plt.plot(grouped_data[a].index[first_point:], y[a][first_point:], '-x',
#                  label=(analysis.varName + ' = ' + str(analysis.variable[a])))
#         plt.xscale('log')
#         plt.legend(loc='best')
#         plt.ylabel(y_label, fontsize=fond_size)
#         plt.xlabel("# data points", fontsize=fond_size)
#         plt.title(plot_title, fontsize=fond_size)
#         plt.grid(True)
#         if yscale_log == True:
#             plt.yscale('log')


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline
# import seaborn as sns
# import pandas as pd

In [ ]:
# analysis = noiseAna
# # move to after analysis was run
# for a in range(0,len(analysis.variable)):
#     analysis.data_list[a] = pd.read_csv(analysis.csvNaming[a]+'.csv', header=None, names=naming)

# plot_dd_ana(analysis, 'L2norm', 'std', 1, fig_size = [6,4])
# plot_dd_ana(analysis, 'L2norm', 'std', first_point = 2)
# plot_dd_ana(analysis, 'L2norm', 'var', first_point = 2)

# # box_ave(noiseAna, 'iterations')
# # box_ave(noiseAna, 'rmsPoiErr')
# # box_ave(noiseAna, 'rmsErr')
# # box_ave(noiseAna, 'rmsErrPerc',1)

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# print(
#     "Loading plot_data(analysis, errorType='rmsErr', grouping='mean', box_plots=0, first_point=0, fig_size=[6.4, 4.8], yscale_log=False, fond_size=15)")

# print("errorType: rmsErr, p_difference, justErr, rmsPoiErr, iterations \ngrouping: mean, std, var \nbox_plots: 1=yes, 0=no \nyscale_log: False, True")


# def plot_dd_ana(analysis, errorType='L2norm', grouping='mean', box_plots=0, first_point=0, fig_size=[6.4, 4.8], yscale_log=False, fond_size=15):
#     grouped_data = [None] * len(analysis.variable)
#     y = [None] * len(analysis.variable)
#     for a in range(0, len(analysis.variable)):

#         if grouping == 'mean':  # taking averages
#             grouped_data[a] = analysis.data_list[a].groupby(['datanum']).mean()
#             plot_title = 'Mean'
#         if grouping == 'std':  # taking standard deviation
#             grouped_data[a] = analysis.data_list[a].groupby(['datanum']).std()
#             plot_title = 'Standard Deviation' # check the terminology
#         if grouping == 'var':  # taking standard deviation
#             grouped_data[a] = analysis.data_list[a].groupby(['datanum']).var()
#             plot_title = 'Variance'
#         if grouping == 'median':  # taking standard deviation
#             grouped_data[a] = analysis.data_list[a].groupby(
#                 ['datanum']).median()
#             plot_title = 'Median'

#         if errorType == 'rmsPoiErr':
#             y[a] = grouped_data[a].rmsPoiErr
#             y_box = analysis.data_list[a].rmsPoiErr
#             y_label = 'Point distance RMS error'
#         # if errorType == 'rmsErrPerc':
#         #     y[a] = grouped_data[a].rmsErrPerc
#         #     y_box = analysis.data_list[a].rmsErrPerc
#         #     y_label = 'Pressure RMS error [%]'
#         if errorType == 'L2norm':
#             y[a] = grouped_data[a].L2norm
#             y_box = analysis.data_list[a].L2norm
#             y_label = 'Temperature RMS error'
#         # if errorType == 'justErr':
#         #     y[a] = grouped_data[a].justErr
#         #     y_box = analysis.data_list[a].justErr
#         #     y_label = 'Accumulated pressure error'
#         # if errorType == 'p_difference':
#         #     y[a] = grouped_data[a].p_difference
#         #     y_box = analysis.data_list[a].p_difference
#         #     y_label = 'Accumulated pressure difference'
#         if errorType == 'iterations':
#             y[a] = grouped_data[a].iterations
#             y_box = analysis.data_list[a].iterations
#             y_label = '# iterations'

#         if box_plots == 1:  # plotting box plots
#             fig = plt.figure(figsize=fig_size)
#             ax1 = sns.boxplot(x="datanum", y=y_box, data=analysis.data_list[a]).set_title(
#                 analysis.varName + ' = ' + str(analysis.variable[a]))
#             plt.show()

#     # fig = plt.figure(figsize=(9, 6))
#     fig = plt.figure(figsize=fig_size)
#     for a in range(0, len(analysis.variable)):
#         plt.plot(grouped_data[a].index[first_point:], y[a][first_point:], '-x',
#                  label=(analysis.varName + ' = ' + str(analysis.variable[a])))
#         plt.xscale('log')
#         plt.legend(loc='best')
#         plt.ylabel(y_label, fontsize=fond_size)
#         plt.xlabel("# data points", fontsize=fond_size)
#         plt.title(plot_title, fontsize=fond_size)
#         plt.grid(True)
#         if yscale_log == True:
#             plt.yscale('log')
